This notebook was built utilising code from the work of Birks in his dissertation, refer to it for more information:

Birks, J. (2023). “Interpretable AI in portfolio management: Ensemble rule models and causal
pruning via large language models”. Master’s dissertation. University of Warwick.

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from concurrent.futures import ProcessPoolExecutor
import time

## ARM

In [ ]:
# Function that ranks data into quintiles
def rank_factors(info):
    for item in info.columns.tolist():
        string = item + " Rank"
        info[string] = pd.qcut(info[item], 5, labels=[item + "1", item + "2", item + "3", item + "4", item + "5"])
    return info

In [ ]:
# Function that creates an array of lists of factors for each available asset
def fpg_prep(info):
    # Drop the columns that do not contain the rankings
    state = info.drop([col for col in info.columns if 'Rank' not in col and pd.api.types.is_numeric_dtype(info[col])], axis=1)
    # Drop the returns ranked column and assign remaining info to new 
    new = state.drop("Return Rank", axis=1)
    # Reset the index of state and drop the names column
    state = state.reset_index()
    final = []
    # For each row, append final with each row as an array of its own
    for i in range(0, len(state)):
        final.append(state.loc[i, state.columns[1:]].tolist())
    # Return both final and new
    return final, new

In [ ]:
# Function that mines association and lift rules, the 5th quintile returns and the true/false df for the FPG algorithm
def rules(final, sup, conf):
    # Preprocessing of input argument into true and false for each discretisation
    te = TransactionEncoder()
    te_ary = te.fit(final).transform(final)
    # Eg is the true/false dataframe in this form for the fp growth algorithm
    eg = pd.DataFrame(te_ary, columns=te.columns_)
    # True/false column of the highest quintile of returns
    high_returns = eg.Return5
    # Finding frequent items in data for a minimum support of 5%
    freq_items = fpgrowth(eg, min_support=sup, use_colnames=True)
    # Discover association and causal rules
    asso_rules = association_rules(freq_items, metric="confidence", min_threshold=conf)
    lift_rules = association_rules(freq_items, metric="lift", min_threshold=1.2)
    # Return true/false dataframe, high returns, and the found
    return eg, high_returns, asso_rules, lift_rules

In [ ]:
# Get high return rules from rule set
def high_ret_rules(asso_rules):
    factors = []
    for index in asso_rules.index.tolist():
        if set(["Return5"]).issubset(set(list(asso_rules.loc[index, 'consequents']))):
            factors.append(list(asso_rules.loc[index, 'antecedents']))
    return factors

In [ ]:
# Function to get rules
def get_rules(unique_asso):
    associatons = []
    for asso in unique_asso:
        if not isinstance(asso, list):
            associatons.append([asso])
        else:
            associatons.append(asso)
    return associatons

In [ ]:
# Function to get rules for the period, by using the above functions
def rules_for_period(info, sup, conf):
    info = rank_factors(info.iloc[:, 2:])
    final, new = fpg_prep(info)
    eg, high_returns, asso_rules, lift_rules = rules(final, sup, conf)
    factors = high_ret_rules(asso_rules)
    
    # Drop only the ranking columns but keep the original data columns
    info = info.drop([col for col in info.columns if 'Rank' not in col and pd.api.types.is_numeric_dtype(info[col])], axis=1)
    
    # Get rule set
    rules_set = get_rules(factors)
    
    # Remove duplicate rules and return them as a list
    unique_rules = list(np.unique(rules_set))
    
    return unique_rules, eg, high_returns, info

## Chi Squared

In [ ]:
from scipy import stats

In [ ]:
# Function to get expected and actual frequency of a rule
def expected_freq(info, rule):
    # Get number of equities with top quintile returns
    num_ret_ind = len(info[info['Return Rank'] == 'Return5'])
    
    # Calculate expected frequency
    mask = info.isin(rule)
    filtered = info[mask].dropna(axis=0, how='all')
    num_rule_ind = len(filtered)
    data_len = len(info)
    ef_ind = (num_ret_ind / data_len) * (num_rule_ind / data_len) * data_len
    
    # Calculate actual frequency
    rule.append('Return5')
    mask = info.isin(rule)
    filtered = info[mask]
    filtered = filtered.dropna(thresh=len(rule))
    actual_freq = len(filtered)
    rule.pop(-1)
    return actual_freq, ef_ind

In [ ]:
# Make list of strings into list of lists, where each string becomes its own list
def list_list(lis):
    
    list_of_lists = []
    
    for string in lis:
        # Create a new list containing the current string
        new_list = [string]
        # Add the new list to the list_of_lists
        list_of_lists.append(new_list)
    
    return list_of_lists

In [ ]:
# Get unique elements for a list
def get_unique_elements(input_list):
    unique_elements = []
    for element in input_list:
        if element not in unique_elements:
            unique_elements.append(element)
    return unique_elements

In [ ]:
# Function to get rules that pass the chi-squared pruning
def causal_chi(info, rules):
    if len(rules) == 0:
        return []
    causal = []
    chi_stats = []
    # Append rules that have a significant Chi-stat
    for rule in rules:
        if type(rule) == np.str_:
            rule = [rule]
        actual_freq, ef_ind = expected_freq(info, rule)
        stat = ((actual_freq - ef_ind) ** 2) / ef_ind
        dof = len(info['Return Rank'].unique()) - 1
        p = stats.chi2.cdf(stat, dof)
        if p > 0.95:
            causal.append(rule)
            chi_stats.append(stat)
    
    if rules == causal or list_list(rules) == causal:
        causal2 = []
        index_1 = chi_stats.index(max(chi_stats))
        causal2.append(causal[index_1])
        chi_stats[index_1] = 0
        
        index_2 = chi_stats.index(max(chi_stats))
        causal2.append(causal[index_2])
        chi_stats[index_2] = 0
        
        index_3 = chi_stats.index(max(chi_stats))
        causal2.append(causal[index_3])
        chi_stats[index_3] = 0
        
        return get_unique_elements(causal2)
        
    return causal

## Simulation

In [ ]:
# Function to find the n most frequent items from a list
def find_n_most_frequent_items(lst, N):
    random.shuffle(lst)
    item_counts = Counter(lst)
    most_common_items = item_counts.most_common()
    N_ind = min((len(most_common_items) - 1), (N - 1))
    max_count = most_common_items[N_ind][1]
    result1 = [item[0] for item in most_common_items if item[1] > max_count]
    result2 = [item[0] for item in most_common_items if item[1] == max_count]
    return result1, result2

In [ ]:
# This function, given a ruleset finds the picks the equities due to those rules
def get_equities_data(rules, ranked_data, cur_data):
    if len(rules) == 0:
        results = [(cur_data['Return'].mean() + 1), len(cur_data)]
        return results
    else:
        ranked_data.reset_index()
        stocks = []
        number_of_rules = len(rules)
        max_stocks = round(len(ranked_data) * 0.25)
        for rule in rules:
            if type(rule) == np.str_:
                rule = [rule]
            mask = ranked_data.isin(rule)
            filtered = ranked_data[mask]
            filtered = filtered.dropna(thresh=len(rule))
            stocks += filtered.index.tolist()
        stocks1, stocks2 = find_n_most_frequent_items(stocks, round(max_stocks))
        cur_data2 = cur_data[cur_data.index.isin(stocks1)]
        cur_data3 = cur_data[cur_data.index.isin(stocks2)]
        if len(stocks1) > 0 and len(stocks2) > 0:
            mean_returns = (cur_data2['Return'].mean() + 1) * (len(stocks1) / max_stocks) + (cur_data3['Return'].mean() + 1) * (1 - (len(stocks1) / max_stocks))
        elif len(stocks1) > 0 and len(stocks2) == 0:
            mean_returns = (cur_data2['Return'].mean() + 1)
        elif len(stocks1) == 0 and len(stocks2) > 0:
            mean_returns = (cur_data3['Return'].mean() + 1)
        results = [mean_returns, max_stocks]
        return results

In [ ]:
def simulation(data, periods, sup, conf):
    dates = data.index.unique()
    column_names = ['chi']  
    returns_df = pd.DataFrame(index=dates[5:-periods], columns=column_names)
    size_df = pd.DataFrame(index=dates[5:-periods], columns=column_names)
    
    half_period_plus_1 = (periods / 2) + 1
    
    for i in range(5, len(dates) - periods):
        # Get rolling window data 
        window_data = data.loc[dates[i:i + periods + 1]]
        
        # Filter window data where more than periods/2 periods are available for each stock
        stock_counts = window_data['Name'].value_counts()
        valid_stocks = stock_counts[stock_counts > half_period_plus_1].index
        window_data = window_data[window_data['Name'].isin(valid_stocks)]
        
        # Get current period data
        current_data = window_data.loc[dates[i + periods]].set_index('ID')
        
        # Drop current period data from window data
        window_data = window_data[window_data.index.isin(dates[i:i + periods])]
        
        # Get the ARM and CRM rulesets
        assoc_rules, eg, ret, info = rules_for_period(window_data, sup, conf)
        
        # Apply Chi-Squared pruning
        chi = causal_chi(info, assoc_rules)
        
        # Encode current data and rank factors
        info2 = rank_factors(current_data.iloc[:, 1:])
        info2 = info2.filter(like='Rank', axis=1)
        
        # Get the returns of each rule set 
        chi_equities = get_equities_data(chi, info2, current_data)
        
        # Store results in dataframes
        returns_df.iloc[i - 5, returns_df.columns.get_loc('chi')] = chi_equities[0]
        size_df.iloc[i - 5, size_df.columns.get_loc('chi')] = chi_equities[1]
    
    return returns_df, size_df

In [ ]:
def benchmark(data, periods):
    dates = data.index.unique()
    column_names = ['chi']
    returns_df = pd.DataFrame(index=dates[5:-periods], columns=column_names)
    size_df = pd.DataFrame(index=dates[5:-periods], columns=column_names)

    half_period_plus_1 = (periods / 2) + 1

    for i in range(5, len(dates) - periods):
        # Get rolling window data
        window_data = data.loc[dates[i:i + periods + 1]]
        
        # Filter window data where more than periods/2 periods are available for each stock
        stock_counts = window_data['Name'].value_counts()
        valid_stocks = stock_counts[stock_counts > half_period_plus_1].index
        window_data = window_data[window_data['Name'].isin(valid_stocks)]
        
        # Get current period data
        current_data = window_data.loc[dates[i + periods]].set_index('ID')
        
        # Drop current period data from window data
        window_data = window_data[window_data.index.isin(dates[i:i + periods])]
        
        # Calculate benchmark return
        benchmark_return = current_data['Return'].mean() + 1
        
        # Store results in dataframes
        returns_df.iloc[i - 5, returns_df.columns.get_loc('chi')] = benchmark_return
        size_df.iloc[i - 5, size_df.columns.get_loc('chi')] = len(current_data)
    
    return returns_df, size_df

## Run the Simulation

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

### Clean the Data

In [ ]:
# Dictionary of loaded datasets
datasets = {
    'raw': pd.read_csv('cleaned_raw.csv', index_col='Date'),
    'extended': pd.read_csv('cleaned_extended.csv', index_col='Date'),
    'featuretools_extended': pd.read_csv('cleaned_featuretools_extended.csv', index_col='Date'),
    'tsfresh_extended': pd.read_csv('cleaned_tsfresh_extended.csv', index_col='Date'),
    'featurewiz_extended': pd.read_csv('cleaned_featurewiz_extended.csv', index_col='Date'),
    'pycaret_extended': pd.read_csv('cleaned_pycaret_extended.csv', index_col='Date')
}

In [ ]:
raw = datasets['raw']
extended = datasets['extended']

raw = raw.drop(columns=['Unnamed: 0'])
extended = extended.drop(columns=['Unnamed: 0'])

datasets['raw'] = raw 
datasets['extended'] = extended

In [ ]:
# Clean nearly constant columns
def drop_cum_columns(df):
    # Drop columns that contain 'CUM' in their names
    columns_to_drop = [col for col in df.columns if 'CUM' in col]
    df = df.drop(columns=columns_to_drop)
    
    return df

# Clean nearly constant columns
def drop_date_columns(df):
    # Drop columns that contain 'Date' in their names
    columns_to_drop = [col for col in df.columns if 'Date' in col]
    df = df.drop(columns=columns_to_drop)
    
    return df

In [ ]:
# Clean problematic columns
ft_ext = datasets['featuretools_extended']

ft_ext = drop_cum_columns(ft_ext)

ft_ext = drop_date_columns(ft_ext)

ft_ext2 = ft_ext.drop(columns=['MODE(metrics_data.ID)', 'MODE(metrics_data.Name)'])
                     
datasets['featuretools_extended'] = ft_ext2

In [ ]:
# Clean problematic columns
tf_ext = datasets['tsfresh_extended']

tf_ext2 = tf_ext.drop(columns=['value__has_duplicate_min', 'value__sum_of_reoccurring_values', 
                              'value__sum_of_reoccurring_data_points', 'value__value_count__value_1', 
                              'value__value_count__value_-1','value__has_duplicate', 'value__longest_strike_below_mean', 
                              'value__count_above_mean', 'value__count_below_mean', 'value__last_location_of_maximum', 'value__first_location_of_maximum', 
                              'value__symmetry_looking__r_0.05', 'value__large_standard_deviation__r_0.2', 'value__number_cwt_peaks__n_1', 'value__number_peaks__n_5', 
                              'value__index_mass_quantile__q_0.2', 'value__index_mass_quantile__q_0.3', 'value__index_mass_quantile__q_0.4', 'value__index_mass_quantile__q_0.6', 
                              'value__index_mass_quantile__q_0.7', 'value__index_mass_quantile__q_0.8','value__index_mass_quantile__q_0.1', 'value__index_mass_quantile__q_0.9',
                              'value__last_location_of_minimum', 'value__first_location_of_minimum', 'value__number_cwt_peaks__n_5', 'value__number_peaks__n_1', 'value__number_peaks__n_3', 
                               'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0', 'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0', 
                               'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2', 'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2', 
                               'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4', 'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4', 
                               'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4', 'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6', 
                               'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6', 'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8', 
                               'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8', 'value__value_count__value_0', 'value__number_crossing_m__m_0', 
                               'value__number_crossing_m__m_-1', 'value__fourier_entropy__bins_3', 'value__fourier_entropy__bins_5', 'value__fourier_entropy__bins_10', 'value__fourier_entropy__bins_100'])
                     
datasets['tsfresh_extended'] = tf_ext2

In [ ]:
# Function to drop low variance columns
def drop_low_variance(df, threshold=0.02):
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Drop low variance columns
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(numeric_df)
    
    # Get the mask of retained columns
    retained_columns = numeric_df.columns[selector.get_support()]
    
    # Retain only the columns that passed the variance threshold
    df_cleaned = df[retained_columns]
    
    # Add back any non-numeric columns
    non_numeric_df = df.select_dtypes(exclude=[np.number])
    df_cleaned = pd.concat([non_numeric_df, df_cleaned], axis=1)
    
    return df_cleaned

In [ ]:
# Apply the function to each dataset
cleaned_datasets = {name: drop_low_variance(df) for name, df in datasets.items()}

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [ ]:
def MI_reduction(df, target_column='Return', k=10):

    # Separate the target variable
    target = df[target_column]
    
    # Separate the 'Name' and 'ID' columns
    non_numeric_columns = df[['Name', 'ID']]
    
    # Separate features and remove non-numeric columns
    features = df.drop(columns=[target_column, 'Name', 'ID'])
    features_numeric = features.select_dtypes(include=[float, int])
    
    # Standardize the features 
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features_numeric)
    
    # Apply SelectKBest 
    selector = SelectKBest(mutual_info_regression, k=k)
    selected_features = selector.fit_transform(scaled_features, target)
    
    # Get the names of the selected features
    selected_feature_names = features_numeric.columns[selector.get_support()]
    
    # Create a DataFrame with the selected features and add the target and non-numeric columns back
    reduced_df = pd.DataFrame(selected_features, columns=selected_feature_names, index=df.index)
    reduced_df[target_column] = target
    reduced_df = pd.concat([non_numeric_columns, reduced_df], axis=1)
    
    return reduced_df

In [ ]:
# Dictionary to store reduced datasets
reduced_datasets = {}

# Number of features to keep for each dataset
features_to_keep = {
    'raw': 10,
    'extended': 20,
    'featuretools_extended': 40,
    'tsfresh_extended': 40,
    'featurewiz_extended': 70,
    'pycaret_extended': 80
}

In [ ]:
# Apply the mutual information reduction to each dataset
for name, df in cleaned_datasets.items():
    k = features_to_keep[name]  
    reduced_df = MI_reduction(df, k=k)
    reduced_datasets[name] = reduced_df

In [ ]:
# Save each reduced dataset to a CSV file
for name, df in reduced_datasets.items():
    df.to_csv(f'reduced_{name}.csv', index=True)

### Apply SAI Method to Each Dataset

In [ ]:
# Dictionary of loaded datasets
reduced_datasets = {
    'raw': pd.read_csv('reduced_raw.csv', index_col='Date'),
    'extended': pd.read_csv('reduced_extended.csv', index_col='Date'),
    'featuretools_extended': pd.read_csv('reduced_featuretools_extended.csv', index_col='Date'),
    'tsfresh_extended': pd.read_csv('reduced_tsfresh_extended.csv', index_col='Date'),
    'featurewiz_extended': pd.read_csv('reduced_featurewiz_extended.csv', index_col='Date'),
    'pycaret_extended': pd.read_csv('reduced_pycaret_extended.csv', index_col='Date')
}

In [ ]:
# Raw dataset
returns_raw, size_raw = simulation(reduced_datasets['raw'], periods=12, sup=0.05, conf=0.20)
returns_raw.to_csv('raw_returns.csv')
size_raw.to_csv('raw_sizes.csv')
print("Raw dataset simulation completed and results saved.")

In [ ]:
# Extended dataset
returns_extended, size_extended = simulation(reduced_datasets['extended'], periods=12, sup=0.05, conf=0.20)
returns_extended.to_csv('extended_returns.csv')
size_extended.to_csv('extended_sizes.csv')
print("Extended dataset simulation completed and results saved.")

In [ ]:
# Featuretools Extended dataset
returns_featuretools_extended, size_featuretools_extended = simulation(reduced_datasets['featuretools_extended'], periods=12, sup=0.05, conf=0.20)
returns_featuretools_extended.to_csv('featuretools_extended_returns.csv')
size_featuretools_extended.to_csv('featuretools_extended_sizes.csv')
print("Featuretools Extended dataset simulation completed and results saved.")

In [ ]:
# TSFRESH Extended dataset
returns_tsfresh_extended, size_tsfresh_extended = simulation(reduced_datasets['tsfresh_extended'], periods=12, sup=0.05, conf=0.20)
returns_tsfresh_extended.to_csv('tsfresh_extended_returns.csv')
size_tsfresh_extended.to_csv('tsfresh_extended_sizes.csv')
print("TSFRESH Extended dataset simulation completed and results saved.")

In [ ]:
# FeatureWiz Extended dataset
returns_featurewiz_extended, size_featurewiz_extended = simulation(reduced_datasets['featurewiz_extended'], periods=12, sup=0.05, conf=0.20)
returns_featurewiz_extended.to_csv('featurewiz_extended_returns.csv')
size_featurewiz_extended.to_csv('featurewiz_extended_sizes.csv')
print("FeatureWiz Extended dataset simulation completed and results saved.")

In [ ]:
# PyCaret Extended dataset
returns_pycaret_extended, size_pycaret_extended = simulation(reduced_datasets['pycaret_extended'], periods=12, sup=0.05, conf=0.20)
returns_pycaret_extended.to_csv('pycaret_extended_returns.csv')
size_pycaret_extended.to_csv('pycaret_extended_sizes.csv')
print("PyCaret Extended dataset simulation completed and results saved.")

In [ ]:
# Obtain the benchmark portfolio returns and size
ret_bench, size_bench = benchmark(datasets['raw'],1)

In [ ]:
# Save benchmark results to CSV
ret_bench.to_csv('ret_bench.csv', index=True)
size_bench.to_csv('siz_bench.csv', index=True)

## Simulate Optimal Set (After running notebook 5)

In [ ]:
optimal = pd.read_csv('Optimal_feature_set.csv', index_col='Date')

In [ ]:
opt_ret_df, opt_size_df = simulation(optimal, 12, 0.05, 0.20)

In [ ]:
opt_ret_df.to_csv('ret_opt.csv', index=True)
opt_size_df.to_csv('siz_opt.csv', index=True)